In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoModelForCausalLM, LlamaTokenizerFast

import torch
from transformers import AutoTokenizer
import os



/home/amax/anaconda3/envs/megatron/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

model1 = LlamaForCausalLM.from_pretrained(
    pretrained_model_name_or_path="/nvme/qd/ckpt/prune_llama2",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    local_files_only=True,
    use_safetensors=False
).to("cuda:5")
tokenizer1 = AutoTokenizer.from_pretrained("/nvme/qd/ckpt/prune_llama2",use_fast=False)
prompt1 = f"""What is the color of the sky?"""
input_ids1 = tokenizer1(prompt1, return_tensors="pt", truncation=True).input_ids.to(model1.device)
print(tokenizer1.sp_model.vocab_size())
print(input_ids1)
print(tokenizer1.batch_decode(input_ids1.detach().cpu().numpy()))

# outputs1 = model1.generate(input_ids=input_ids1, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

# print(f"Generated instruction:\n{tokenizer1.batch_decode(outputs1, skip_special_tokens=True)[0]}")

Loading checkpoint shards: 100%|██████████| 3/3 [00:14<00:00,  4.82s/it]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


15258
tensor([[    1,  1516,   337,   277,  2493,   309,   277,  9490, 15246]],
       device='cuda:5')
['<s>What is the color of the sky?']


tensor([[    1,  1516,   337,   277,  2493,   309,   277,  9490, 15246, 10713,
         11781,  6505,  4692, 12993, 12530,  9883, 11607,  6163,  3435, 12791,
          6473, 10041,  8222,  6557,  4542, 13688,  6508,  7566,  8899, 11812,
         13957, 10188,  5609, 12241,  4079,  8041,  5145,  9123,  9820, 13632,
          3552,  2482,  5406,  9547,  9815, 14900,  2570,  8298, 13387, 11695,
          6758,  3002,  3278,  5238,  3874,  6124, 11863, 13536, 12312,  9098,
         12517,  7105,  6616, 12032,  7689, 10623, 14795, 13819, 13640, 12441,
          5940,  5373,  4250,  7454,  5904,  4325,  1932,  8887,  6773,  8562,
         12865,  4969, 14077,  7315, 11117, 11748,  6435,  8581, 10163,  3583,
         14056, 12141,  4072,  2377,  3266,  5645, 14896,  7188,  9070, 15194,
          7091,  4185, 13914,  8166, 11210,  3609, 14698,  1851, 10340]],
       device='cuda:5')


In [4]:

model2 = LlamaForCausalLM.from_pretrained(
    pretrained_model_name_or_path="/nvme/hf_models/Llama-2-7b-hf",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    local_files_only=True,
    use_safetensors=False
).to("cuda:5")
tokenizer2 = AutoTokenizer.from_pretrained("/nvme/hf_models/Llama-2-7b-hf",use_fast=False)
prompt2 = f"""What is the color of the sky?"""
input_ids2 = tokenizer2(prompt2, return_tensors="pt", truncation=True).input_ids.to(model1.device)
print(tokenizer2.sp_model.vocab_size())
print(input_ids2)
print(tokenizer2.batch_decode(input_ids2.detach().cpu().numpy()))

Loading checkpoint shards: 100%|██████████| 3/3 [00:14<00:00,  4.80s/it]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


32000
tensor([[    1,  1724,   338,   278,  2927,   310,   278, 14744, 29973]],
       device='cuda:5')
['<s>What is the color of the sky?']


In [8]:
output1 = model1(input_ids1).logits
output2 = model2(input_ids2).logits


In [10]:
print(output1)
print(output2)

tensor([[[-6.9922, -6.9922, -6.9922,  ..., -6.9922, -6.9922, -6.9922],
         [-6.9922, -6.9922, -6.9922,  ..., -6.9922, -6.9922, -6.9922],
         [-6.9922, -6.9922, -6.9922,  ..., -6.9922, -6.9922, -6.9922],
         ...,
         [-6.9922, -6.9922, -6.9922,  ..., -6.9922, -6.9922, -6.9922],
         [-6.9922, -6.9922, -6.9922,  ..., -6.9922, -6.9922, -6.9922],
         [-6.9922, -6.9922, -6.9922,  ..., -6.9922, -6.9922, -6.9922]]],
       device='cuda:5', grad_fn=<ToCopyBackward0>)
tensor([[[-12.8281,  -7.4453,  -0.4639,  ...,  -6.7812,  -8.0156,  -7.5039],
         [-12.6016,  -9.4531,  -2.6113,  ...,  -7.0469,  -9.5156,  -9.2422],
         [-13.0625, -15.0000,  -3.6855,  ...,  -6.0625,  -8.1484,  -8.5078],
         ...,
         [ -9.8125,  -9.0703,   0.5933,  ...,  -6.9336,  -6.3359,  -6.5039],
         [ -4.5469,  -3.0176,   9.6094,  ...,  -2.8027,  -3.2480,  -3.2344],
         [-12.0703, -10.5781,   6.9219,  ...,  -5.9609,  -7.5508,  -5.8828]]],
       device='cuda:5', grad_

In [12]:
print(input_ids1)
print(input_ids2)
print(model1.model.embed_tokens.weight[1].sum())
print(model2.model.embed_tokens.weight[1].sum())

tensor([[    1,  1516,   337,   277,  2493,   309,   277,  9490, 15246]],
       device='cuda:5')
tensor([[    1,  1724,   338,   278,  2927,   310,   278, 14744, 29973]],
       device='cuda:5')
Parameter containing:
tensor([[ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
         -6.5565e-06,  8.9407e-07],
        [ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
         -6.5565e-06,  8.9407e-07],
        [ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
         -6.5565e-06,  8.9407e-07],
        ...,
        [ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
         -6.5565e-06,  8.9407e-07],
        [ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
         -6.5565e-06,  8.9407e-07],
        [ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
         -6.5565e-06,  8.9407e-07]], device='cuda:5', dtype=torch.float16,
       requires_grad=True)
Parameter containing:
tensor([[ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-

In [2]:
import sentencepiece as spm
sp_model = spm.SentencePieceProcessor(model_file="/nvme/qd/ckpt/prune_llama2/tokenizer.model")

In [6]:
print(sp_model.vocab_size())
print(sp_model.Encode("D"))

15258
[359]


: 